In [ ]:
# %load_ext autoreload
# %autoreload 2

In [1]:
###### check two data set
import os, sys
sys.path.append("/home/n/nguyenpk/CS6220/otdd")
import otdd
from otdd.pytorch.distance import DatasetDistance
#
import pandas as pd
import math
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torch.nn.utils.rnn import pad_sequence
import pickle, pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
import torch.nn as nn

import torch
import numpy as np
#----

from transformers import BertModel, BertTokenizer
import torch # the main pytorch library
import torch.nn.functional as f
#---
seed = 100
if torch.cuda.is_available():
    generator = torch.Generator('cuda').manual_seed(seed)
else:
    generator = torch.Generator().manual_seed(seed)

/home/n/nguyenpk/CS6220/otdd/otdd/pytorch/distance.py:23: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm



[KeOps] Warning : There were warnings or errors compiling formula :
<stdin>:1:10: fatal error: cuda.h: No such file or directory
compilation terminated.

[KeOps] Warning : 
    The location of Cuda header files cuda.h and nvrtc.h could not be detected on your system.
    You must determine their location and then define the environment variable CUDA_PATH,
    either before launching Python or using os.environ before importing keops. For example
    if these files are in /vol/cuda/10.2.89-cudnn7.6.4.38/include you can do :
      import os
      os.environ['CUDA_PATH'] = '/vol/cuda/10.2.89-cudnn7.6.4.38'
    
[KeOps] Compiling cuda jit compiler engine ... 
[KeOps] Warning : There were warnings or errors compiling formula :
/home/n/nguyenpk/.local/lib/python3.8/site-packages/keopscore/binders/nvrtc/nvrtc_jit.cpp:5:10: fatal error: nvrtc.h: No such file or directory
    5 | #include <nvrtc.h>
      |          ^~~~~~~~~
compilation terminated.

OK
[pyKeOps] Compiling nvrtc binder for pytho

ot.gpu not found - coupling computation will be in cpu


In [5]:
dict_DS = {} ## --> data_name -> {'all', 'train', 'test', 'dev'}
rs = {}
# Read data from file and process
for ikey in ["IEMOCAP", "Daily", "MELD"]:
    dict_DS[ikey] = pd.read_pickle("data/Glove_embedding_{}_token_fts.pkl".format(ikey)) 
    idata = dict_DS[ikey]
    ll_x = []
    ll_y = []
    
    for i in ["train", "test", "dev"]:
        i_seg_data = torch.cat(idata[i]["data"], dim=0)
        i_seg_label =  torch.tensor(np.concatenate(idata[i]["emotion"]), dtype=int)
        rs["{}_{}".format(ikey, i)] = (i_seg_data, i_seg_label)
        ll_x.append(i_seg_data)
        ll_y.append(i_seg_label)
    rs["{}_all".format(ikey)] = (torch.cat(ll_x, dim=0), torch.cat(ll_y, dim=0))


In [6]:
class my_DataSet(Dataset):
    """
     This class create my dataset
    """
    def __init__(self, data):
        self.data = data[0]
        self.targets = data[1]
        self.len = len(self.targets)
    def __getitem__(self, index):
        return self.data[index], self.targets[index]
    def __len__(self):
        return self.len
    def __delitem__(self, key):
        self.data = np.delete(self.data, key, axis=0)
        self.targets = np.delete(self.targets, key, axis=0)

In [7]:
import os
os.environ['CUDA_PATH'] = '/usr/local/cuda-11'
print("Create the dataset")
dict_myDS = {}
for i in rs.keys():
    dict_myDS[i] = my_DataSet(rs[i])
    print(i, len(dict_myDS[i]), dict_myDS[i].data.shape, dict_myDS[i].targets.shape)

print("Begin compute the distance")
mapping_dict = dict(zip(range(len(dict_myDS)), list(dict_myDS.keys())))
m_matrix = np.zeros([len(mapping_dict), len(mapping_dict)])
for i in mapping_dict.keys():
    for j in mapping_dict.keys():
        print("Begin compute distance between:", mapping_dict[i], mapping_dict[j])
        D_i = dict_myDS[mapping_dict[i]]
        D_j = dict_myDS[mapping_dict[j]]
        dist = DatasetDistance(D_i, D_j,
                                   inner_ot_method = 'gaussian_approx',#gaussian_approx
                                   debiased_loss = True,
                                   p = 2, entreg = 1e-1,
                                   min_labelcount=1,
                                   device='cpu') ## error device!!
        
        d = dist.distance(maxsamples = 1000)
        m_matrix[i,j]  = d.item()
        print("Finish compute distance:", m_matrix[i,j])
print("Begin to saving")

distance = pd.DataFrame(m_matrix, columns=mapping_dict.values(), index=mapping_dict.values())
rs_glove_embedding_OTDD = {'df': distance, 'idx': mapping_dict, 'matrix':m_matrix}
pd.to_pickle(rs_glove_embedding_OTDD, "data/result/rs_glove_embedding_OTDD.pkl")

Create the dataset
IEMOCAP_train 5146 torch.Size([5146, 300]) torch.Size([5146])
IEMOCAP_test 1623 torch.Size([1623, 300]) torch.Size([1623])
IEMOCAP_dev 664 torch.Size([664, 300]) torch.Size([664])
IEMOCAP_all 7433 torch.Size([7433, 300]) torch.Size([7433])
Daily_train 87170 torch.Size([87171, 300]) torch.Size([87170])
Daily_test 7740 torch.Size([7740, 300]) torch.Size([7740])
Daily_dev 8069 torch.Size([8069, 300]) torch.Size([8069])
Daily_all 102979 torch.Size([102980, 300]) torch.Size([102979])
MELD_train 9989 torch.Size([9989, 300]) torch.Size([9989])
MELD_test 2610 torch.Size([2610, 300]) torch.Size([2610])
MELD_dev 1109 torch.Size([1109, 300]) torch.Size([1109])
MELD_all 13708 torch.Size([13708, 300]) torch.Size([13708])
Begin compute the distance
Begin compute distance between: IEMOCAP_train IEMOCAP_train


/home/n/nguyenpk/CS6220/otdd/otdd/pytorch/moments.py:173: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2318.)
  return self.μ, self.Σ - torch.ger(self.μ.T,self.μ)*self.n/(self.n-1)


0it [00:00, ?it/s]

/home/n/nguyenpk/CS6220/otdd/otdd/pytorch/sqrtm.py:54: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2524.)
  s, v = A.symeig(eigenvectors=True) # This is faster in GPU than CPU, fails gradcheck. See https://github.com/pytorch/pytorch/issues/30578


  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Create DD topics!
- Can move this part to another place later


In [21]:
from preprocessing import preprocessing_raw as pr
from preprocessing import preprocessing_token as pt
#
import sys
from sysconf import  conf
import pandas as pd, numpy as np, pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset


In [22]:
daily_tool = pr.Daily_Dialog_preprocessing()
conv_path = os.path.join(conf["raw_Daily_Dailog_path"], "dialogues_text.txt")
emo_path = os.path.join(conf["raw_Daily_Dailog_path"], "dialogues_emotion.txt")
act_path = os.path.join(conf["raw_Daily_Dailog_path"], "dialogues_act.txt")
topic_path = os.path.join(conf["raw_Daily_Dailog_path"], "dialogues_topic.txt")
data_raw_DD_topic = daily_tool.reading_DD(conv_path, emo_path, act_path, topic_path)
############ --- create again to make sure all of the data is have same label


In [125]:
len(data_raw_DD_topic['actions'])

13118

In [129]:
ll_d = []
ll_l = []
ll_t = []
ll_a = []
for i in  range(len(data_raw_DD_topic['conversation'])):
    data = data_raw_DD_topic['conversation'][i]
    label = data_raw_DD_topic['emotions'][i]
    topic = data_raw_DD_topic['topics'][i]
    action = data_raw_DD_topic['actions'][i]
    if len(data) != len(label): ## cancel this data
        continue
    ll_d.append(data)
    ll_l.append(label)
    ll_t.append(topic)
    ll_a.append(action)
data_raw_DD_topic = {'conversation': ll_d, 'emotions': ll_l, "actions":ll_a, "topics":ll_t}

pd.to_pickle(data_raw_DD_topic, "data/Daily_topic_raw.pkl")

In [23]:
## create the tokenize
data_raw_DD_topic = pd.read_pickle("data/Daily_topic_raw.pkl")

tokenizer = Tokenizer()
tokenizer.fit_on_texts(pt.flatten_list(data_raw_DD_topic['conversation']))
max_num_tokens = 300
glove_vector = pt.load_pretrained_glove()
##
word_vector_length = len(glove_vector['the'])
word_index = tokenizer.word_index
inv_word_index = {v: k for k, v in word_index.items()}
num_unique_words = len(word_index)
glv_embedding_matrix = np.zeros((num_unique_words + 1, word_vector_length))
for j in range(1, num_unique_words + 1):
    try:
        glv_embedding_matrix[j] = glove_vector[inv_word_index[j]]
    except KeyError:
        glv_embedding_matrix[j] = np.random.randn(word_vector_length) / 200

Loading GloVe model, this can take some time...


In [9]:
data_token = pt.create_data_token(data_raw_DD_topic, tokenizer, max_num_tokens)
# pd.to_pickle(data_token, "data/Daily_topic_token.pkl")

In [42]:
#-----------
ll = []
ll_e = []
ll_t =[]
rs = {}
vocab_size, e_dim = glv_embedding_matrix.shape
embedding = nn.Embedding(vocab_size,e_dim)
embedding.weight = nn.Parameter(torch.from_numpy(glv_embedding_matrix).float())
for idx, i in  enumerate(data_token['data_token']):
    if len(i) >= 12:
        if data_token["topics"][idx] == 4:
            continue
        ll.append(embedding(torch.tensor(i, dtype=int)).sum(axis=1))
        ll_e.append(data_token["emotions"][idx])
        ll_t.append(data_token["topics"][idx])
        
rs["data"] = ll
rs["emotion"] = ll_e
rs["topics"] = ll_t

pd.to_pickle(rs, "data/Glove_embedding_Daily_topic.pkl")

In [40]:
len(rs["topics"] )

2534

In [41]:
[ sum(rs["topics"] == i) for i in np.unique(rs["topics"])]

[586, 1392, 556]

In [43]:
from sklearn import preprocessing
class my_DataSet(Dataset):
    """
     This class create my dataset
    """
    def __init__(self, data):
        self.data = data[0]
        self.clf = preprocessing.LabelEncoder()
        self.targets = torch.tensor(self.clf.fit_transform(data[1]), dtype=int)
        self.y_raw = data[1] 
        self.len = len(self.targets)
    def __getitem__(self, index):
        return self.data[index], self.targets[index]
    def __len__(self):
        return self.len
    def __delitem__(self, key):
        self.data = np.delete(self.data, key, axis=0)
        self.targets = np.delete(self.targets, key, axis=0)

In [1]:
rs = pd.read_pickle("data/Glove_embedding_Daily_topic.pkl")
topic_DS = {}
for i in sorted(np.unique(rs["topics"])):
    ll_x = []
    ll_y = []
    for j in rs["topics"]:
        data = rs['data'][j]
        label = rs['emotion'][j]
        ll_x.append(data)
        ll_y.append(label)
    X = torch.cat(ll_x)
    y = torch.tensor(np.concatenate(ll_y), dtype=int)
    topic_DS[i] = my_DataSet((X,y))

NameError: name 'pd' is not defined

In [177]:
mapping_dict = dict(zip(range(len(topic_DS)), list(topic_DS.keys())))
m_matrix = np.zeros([len(topic_DS.keys()), len(topic_DS.keys())])
for i in range(len(topic_DS.keys())):
    for j in range(len(topic_DS.keys())):
        print("Begin compute distance between:", i, j)
        D_i = topic_DS[mapping_dict[i]]
        D_j = topic_DS[mapping_dict[j]]
        dist = DatasetDistance(D_i, D_j,
                                   inner_ot_method = 'gaussian_approx',#gaussian_approx
                                   debiased_loss = True,
                                   p = 2, entreg = 1e-1,
                                   min_labelcount=1,
                                   device='cpu') ## error device!!
        
        d = dist.distance(maxsamples = 1000)
        m_matrix[i,j]  = d.item()
        print("Finish compute distance:", m_matrix[i,j])
print("Begin to saving")

pd.to_pickle(m_matrix, "data/result/rs_glove_TOPIC_OTDD.pkl")

Begin compute distance between: 0 0


0it [00:00, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Finish compute distance: 17.450740814208984
Begin to saving


In [23]:
!ls

00_preprocessing.ipynb	data			   preprocessing  sysconf.py
01_OTDD_glove.ipynb	DD_conv_distance_topic.py  __pycache__
base_method.ipynb	my_DD_ds.pickle		   README.md


In [45]:
################## --- using distance to predict the topics!!!
rs = pd.read_pickle("data/Glove_embedding_Daily_topic.pkl")
DD_DS = {}
for i in range(len(rs['data'])):
    idata = rs['data'][i]
    ilabel = rs['emotion'][i]
    DD_DS[i] = my_DataSet((idata, ilabel))
pd.to_pickle(DD_DS, "my_DD_ds.pickle")

In [46]:
len(DD_DS)

2534

In [2]:
m_matrix = np.zeros([len(DD_DS.keys()), len(DD_DS.keys())])

for i in range(len(DD_DS.keys())):
    for j in range(len(DD_DS.keys())):
        print("Begin compute distance between:", i, j)
        D_i = DD_DS[i]
        D_j = DD_DS[j]
        dist = DatasetDistance(D_i, D_j,
                                   inner_ot_method = 'gaussian_approx',#gaussian_approx
                                   debiased_loss = True,
                                   p = 2, entreg = 1e-1,
                                   min_labelcount=1,
                                   device='cpu') ## error device!!
        try:
            d = dist.distance()
            m_matrix[i,j]  = d.item()
        except:
            continue
        print("Finish compute distance:", m_matrix[i,j])
print("Begin to saving")

pd.to_pickle(m_matrix, "data/result/rs_glove_DISTANCE_topic_conv.pkl")

NameError: name 'np' is not defined

In [24]:
len(DD_DS)

13117